In [ ]:
import os.path

## Automata files managment

In [ ]:
def convert_ats_to_hoa_gff(aut):
    '''Convert an automaton `aut` in `.ats` format into `.hoa`
    and `.gff` formats'''
    command = \
'parseAutomata("{0}.ats");\n\
write(nwa, "{0}", "HOA");\n\
write(nwa, "{0}", "GFF");'
    
    if os.path.exists(aut+'.hoa') and os.path.exists(aut+'.gff'):
        return
    cm = command.format(aut)
    print(cm,file=open('translate.ats','w'))
    !./AutomataScriptInterpreter.sh translate.ats > /dev/null 2>/dev/null
    !rm translate.ats
    # Ultimate inconsistently names AP for .hoa and .gff
    !sed -i "/AP/,/Acceptance/ s/p\([0-9]\+\)/\1/g" {aut}.hoa

In [ ]:
def par(path):
    '''Return a path enriched with skip to parent directory.
    This is used in the recursive Makefiles'''
    return os.path.join(os.pardir,path)

In [ ]:
def is_computed(path):
    '''Some automata files exist, but are empty'''
    return os.path.exists(path) and not os.stat(path).st_size == 0

In [ ]:
def get_cases(dir='.',reset=False,prefix='semideterministic',cs=[]):
    if not cs or reset:
        cs.clear()
        unique_cases = !autfilt --unique --stats=%f {dir}/{prefix}/*.hoa
        cs.extend([os.path.splitext(os.path.basename(case))[0] for case in unique_cases])
    return cs

def get_orig_automaton(case):
    return os.path.join("semideterministic",case+".hoa")

def get_complements(case):
    files = !ls complement/{case}*.hoa
    return files

def get_algorithm_name(aut_file):
    if aut_file[-4:] != ".hoa":
        return "case"
    division = aut_file[:-4].split("__Complement")
    if len(division) == 1:
        return "input"
    return division[1]

In [ ]:
GOAL_algorithms = [
    {'name':'ThreeStep',
     'args':'sdbw'},
    {'name':'SafraPiterman',
     'args':'piterman -macc -sim -eq'},
    {'name':'Rank',
     'args':'rank -macc -tr -ro -cp'},
    {'name':'Slice',
     'args':'slice -macc -eg -madj -ro'},
    {'name':'Ramsey',
     'args':'ramsey -macc -min'},
]

In [ ]:
def get_complement_names(case):
    suffixes = ['__ComplementUltimateBS']
    suffixes.extend(['__ComplementGoal'+alg['name'] for alg in GOAL_algorithms])
    return [case+suf+'.hoa' for suf in suffixes]

###  Generation of simplified automata

In [ ]:
def get_simplified_name(aut_file,degree='high'):
    return "{}_{}.hoa".format(os.path.basename(aut_file)[0:-4],degree)

def get_simplified_path(aut_file,degree='high'):
    ### get the name of the file
    name = get_simplified_name(os.path.basename(aut_file),degree)
    ### add the directory prefix
    return os.path.join('simplified',name)

def get_simplify_command(aut,output_file,degree='high',timeout='5m'):
    return 'timeout {} autfilt --small --{} -B -H {} > {}'.\
        format(timeout,degree,aut,output_file)

### Products for correctness checks

In [ ]:
def get_products(compl):
    prod = os.path.join('products','prod_{}'.format(os.path.basename(compl)))
    comp_prod = os.path.join('products','comp_prod_{}'.format(os.path.basename(compl)))
    return prod,comp_prod

In [ ]:
def get_validate_name(case):
    return 'val_{}.hoa'.format(case)

def get_validate_path(case):
    return os.path.join('products',get_validate_name(case))

In [ ]:
def get_ltl2dstar_name(aut,simplified=False):
    return 'ltl2dstar_'+os.path.basename(aut)

def get_ltl2dstar_path(aut):
    return os.path.join('ltl2dstar',get_ltl2dstar_name(aut))